Los autocodificadores son redes neuronales artificiales capaces de aprender representaciones densas de los datos de entrada, llamadas representaciones o codificaciones latentes, sin ninguna supervisión (es decir, el conjunto de entrenamiento no está etiquetado). Estas codificaciones suelen tener una dimensionalidad mucho más baja que los datos de entrada, lo que hace que los autocodificadores sean útiles para la reducción de la dimensionalidad (ver Capítulo 8), especialmente para fines de visualización. Los autocodificadores también actúan como detectores de características, y se pueden utilizar para el preentrenamiento no supervisado de redes neuronales profundas (como discutimos en el capítulo 11). Por último, algunos autocodificadores son modelos generativos: son capaces de generar aleatoriamente nuevos datos que se parecen mucho a los datos de entrenamiento. Por ejemplo, podrías entrenar a un autocodificador en imágenes de caras, y entonces podría generar nuevas caras.

Las redes adversarias generativas (GAN) también son redes neuronales capaces de generar datos. De hecho, pueden generar imágenes de caras tan convincentes que es difícil creer que las personas que representan no existan. Puedes juzgarlo por ti mismo visitando https://thispersondoesnotexist.com, un sitio web que muestra caras generadas por una arquitectura GAN llamada StyleGAN. También puedes consultar https://thisrentaldoesnotexist.com para ver algunos anuncios generados de Airbnb. Los GAN ahora se utilizan ampliamente para la súper resolución (aumentar la resolución de una imagen), la coloración, la potente edición de imágenes (por ejemplo, reemplazar los bombarderos fotográficos con un fondo realista), convertir bocetos simples en imágenes fotorrealistas, predecir los siguientes fotogramas en un vídeo, aumentar un conjunto de datos (para entrenar a otros modelos), generar otros tipos de datos (como texto, audio y series temporales), identificar las debilidades en otros modelos para fortalecerlos y más.

Una adición más reciente al grupo de aprendizaje generativo son los modelos de difusión. En 2021, lograron generar imágenes más diversas y de mayor calidad que las GAN, a la vez que fueron mucho más fáciles de entrenar. Sin embargo, los modelos de difusión son mucho más lentos de ejecutar.

Los autocodificadores, los GAN y los modelos de difusión no están supervisados, todos aprenden representaciones latentes, todos se pueden utilizar como modelos generativos y tienen muchas aplicaciones similares. Sin embargo, funcionan de manera muy diferente:

- Los autocodificadores simplemente aprenden a copiar sus entradas a sus salidas. Esto puede sonar como una tarea trivial, pero como verás, restringir la red de varias maneras puede hacerlo bastante difícil. Por ejemplo, puede limitar el tamaño de las representaciones latentes, o puede agregar ruido a las entradas y entrenar a la red para recuperar las entradas originales. Estas restricciones impiden que el autoencoder copie trivialmente las entradas directamente a las salidas, lo que lo obliga a aprender formas eficientes de representar los datos. En resumen, las codificaciones son subproductos del autocodificador que aprende la función de identidad bajo algunas restricciones.

* Los GAN se componen de dos redes neuronales: un generador que intenta generar datos que se parecen a los datos de entrenamiento, y un discriminador que trata de distinguir datos reales de datos falsos. Esta arquitectura es muy original en el aprendizaje profundo en el sentido de que el generador y el discriminador compiten entre sí durante el entrenamiento: el generador a menudo se compara con un criminal que intenta hacer dinero falso realista, mientras que el discriminador es como el investigador de la policía que intenta distinguir el dinero real de la falsificación. El entrenamiento adversario (entrenamiento de redes neuronales competidoras) es ampliamente considerada una de las innovaciones más importantes de la década de 2010. En 2016, Yann LeCun incluso dijo que era "la idea más interesante de los últimos 10 años en el aprendizaje automático".

- Un modelo probabilístico de difusión de desnuido (DDPM) está entrenado para eliminar un poco de ruido de una imagen. Si luego tomas una imagen completamente llena de ruido gaussiano y ejecutas repetidamente el modelo de difusión en esa imagen, surgirá gradualmente una imagen de alta calidad, similar a las imágenes de entrenamiento (pero no idéntica).


En este capítulo comenzaremos explorando con más profundidad cómo funcionan los autocodificadores y cómo usarlos para la reducción de la dimensionalidad, la extracción de características, el preentrenamiento no supervisado o como modelos generativos. Esto naturalmente nos llevará a los GAN. Construiremos un simple GAN para generar imágenes falsas, pero veremos que la formación suele ser bastante difícil. Discutiremos las principales dificultades que encontrará con el entrenamiento adversario, así como algunas de las principales técnicas para evitar estas dificultades. Y, por último, construiremos y entrenaremos un DDPM y lo usaremos para generar imágenes. ¡Comencemos con los autocodificadores!


# Representaciones de datos eficientes

¿Cuál de las siguientes secuencias de números te resulta más fácil de memorizar?

40, 27, 25, 36, 81, 57, 10, 73, 19, 68
50, 48, 46, 44, 42, 40, 38, 36, 34, 32, 30, 28, 26, 24, 22, 20, 18, 16, 14
A primera vista, parece que la primera secuencia debería ser más fácil, ya que es mucho más corta. Sin embargo, si miras cuidadosamente la segunda secuencia, te darás cuenta de que es solo la lista de números pares de 50 a 14. Una vez que notas este patrón, la segunda secuencia se vuelve mucho más fácil de memorizar que la primera porque solo necesitas recordar el patrón (es decir, los números pares decrecientes) y los números iniciales y finales (es decir, 50 y 14). Ten en cuenta que si pudieras memorizar rápida y fácilmente secuencias muy largas, no te importaría mucho la existencia de un patrón en la segunda secuencia. Aprenderías cada número de memoria, y eso sería todo. El hecho de que sea difícil memorizar secuencias largas es lo que hace que sea útil reconocer patrones, y espero que esto aclare por qué restringir un autocodificador durante el entrenamiento lo empuja a descubrir y explotar patrones en los datos.

La relación entre la memoria, la percepción y la coincidencia de patrones fue estudiada por William Chase y Herbert Simon⁠1 a principios de la década de 1970. Observaron que los jugadores expertos de ajedrez fueron capaces de memorizar las posiciones de todas las piezas de un juego mirando el tablero durante solo cinco segundos, una tarea que la mayoría de la gente encontraría imposible. Sin embargo, este fue solo el caso cuando las piezas se colocaban en posiciones realistas (de juegos reales), no cuando las piezas se colocaban al azar. Los expertos en ajedrez no tienen una memoria mucho mejor que tú y yo; solo ven los patrones de ajedrez más fácilmente, gracias a su experiencia con el juego. Los patrones de la nota les ayuda a almacenar información de manera eficiente.

Al igual que los jugadores de ajedrez en este experimento de memoria, un autocodificador mira las entradas, las convierte en una representación latente eficiente y luego escupe algo que (con suerte) se ve muy cerca de las entradas. Un autocodificador siempre se compone de dos partes: un codificador (o red de reconocimiento) que convierte las entradas en una representación latente, seguido de un decodificador (o red generativa) que convierte la representación interna en las salidas (ver Figura 17-1).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1701.png)

(_Figura 17-1. El experimento de memoria de ajedrez (izquierda) y un autocodificador simple (derecha)_)

Como puede ver, un autocodificador suele tener la misma arquitectura que un perceptrón multicapa (MLP; véase el capítulo 10), excepto que el número de neuronas en la capa de salida debe ser igual al número de entradas. En este ejemplo, solo hay una capa oculta compuesta por dos neuronas (el codificador) y una capa de salida compuesta por tres neuronas (el decodificador). Las salidas a menudo se llaman reconstrucciones porque el autocodificador intenta reconstruir las entradas. La función de costo contiene una pérdida de reconstrucción que penaliza el modelo cuando las reconstrucciones son diferentes de las entradas.

Debido a que la representación interna tiene una dimensionalidad más baja que los datos de entrada (es 2D en lugar de 3D), se dice que el autoencoder está incompleto. Un autocodificador incompleto no puede copiar trivialmente sus entradas a las codificaciones, sin embargo, debe encontrar una manera de generar una copia de sus entradas. Se ve obligado a aprender las características más importantes de los datos de entrada (y a dejar de lado las que no son importantes).

Veamos cómo implementar un autocodificador subcompleto muy simple para reducir la dimensionalidad.


# Realizar PCA con un autocodificador lineal incompleto

Si el autocodificador utiliza solo activaciones lineales y la función de costo es el error al cuadrado medio (MSE), entonces termina realizando un análisis de componentes principales (PCA; ver Capítulo 8).

El siguiente código construye un autocodificador lineal simple para realizar PCA en un conjunto de datos 3D, proyectándolo en 2D: